In [4]:
# Import the necessary package
import numpy as np
import pandas as pd
from sklearn.utils import resample
import random
from decimal import Decimal
from IPython.display import display
import warnings

In [5]:
random.seed(666)# set the seed for reproducible reason

#import the data and calculate the mean effort for each treatment
df = pd.read_stata('D:\\Temp\\mturk_clean_data_short.dta')
emp_moments = np.array(np.round(df.groupby("treatment").mean("buttonpresses")))

In [6]:
# Treatments are as follows:
# 1.1: benchmark specification with piece rate of 0.01 
# 1.2: benchmark specification with piece rate of 0.10 
# 1.3: benchmark specification with piece rate of 0.00 
# 3.1: social preferences (charity) with piece rate of 0.01 
# 3.2: social preferences (charity) with piece rate of 0.10 
# 10 : social preferences (gift exchange) bonus of 40 cents (independently of nr buttonpresses)
# 4.1: time discounting with extra 0.01 paid two weeks later
# 4.2: time discounting with extra 0.01 paid four weeks later

# resample is a useful command from the sklearn library that samples with replacement our data.
# I first get a smaller dataframe containing only observations for a specific treatment, I then resample the observations and
# compute the rounded mean of buttonpresses, save the result and then pass onto the next treatment. I do this for 'number' times.
def mybootstrap(dataset, number):
    
    E11, E12, E13, E31, E32, E10, E41, E42 = [], [], [], [], [], [], [], []
    box={'1.1':E11,'1.2':E12,'1.3':E13,'3.1':E31,'3.2':E32,'10':E10,'4.1':E41,'4.2':E42}
    
    for i in range(1,number+1):
        for a,b in box.items():
            db = dataset[dataset.treatment==a]
            bootsample = resample(db['buttonpresses'],replace=True,)
            b.append(np.round(np.mean(bootsample)))
    return E11, E12, E13, E31, E32, E10, E41, E42

E11, E12, E13, E31, E32, E10, E41, E42 = mybootstrap(df,2000)




In [7]:
import numpy as np

def mymindisest(params):
    '''
    params: params is the dictionary include the mean effort value for each bootstrap result or the original bootstrap outcome(so the length can be 2000 for each treatment).
            Besides, it also include a key called "specification", we can choose "Exp" or "Power" as our cost function.
    '''
    # Define constants payoff p
    P = [0, 0.01, 0.1] # P is a vector containing the different piece-rates
    expr = {
        'Exp': {
            'log_k': lambda E11, E12: (np.log(P[2]) - np.log(P[1]) * E12 / E11) / (1 - E12 / E11),
            'log_gamma': lambda log_k, E11: np.log((np.log(P[1]) - log_k) / E11),
            'log_s': lambda log_gamma, E13, log_k: np.exp(log_gamma) * E13 + log_k,
            'EG31': lambda E31, g: np.exp(E31 * g),'EG32': lambda E32, g: np.exp(E32 * g),'EG10': lambda E10, g: np.exp(E10 * g),'EG41': lambda E41, g: np.exp(E41 * g),'EG42': lambda E42, g: np.exp(E42 * g),
        


        },
        'Power': {
            'log_k': lambda E11, E12: (np.log(P[2]) - np.log(P[1]) * np.log(E12) / np.log(E11)) / (1 - np.log(E12) / np.log(E11)),
            'log_gamma': lambda log_k, E11: np.log((np.log(P[1]) - log_k) / np.log(E11)),
            'log_s': lambda log_gamma, E13, log_k: np.exp(log_gamma) * np.log(E13) + log_k,
            'EG31': lambda E31, g: E31 ** g,'EG32': lambda E32, g: E32 ** g,'EG10': lambda E10, g: E10 ** g,'EG41': lambda E41, g: E41 ** g,'EG42': lambda E42, g: E42 ** g
        }
    }

    # Extract arguments from params dictionary
    E11 = np.array(params['E11'])
    E12 = np.array(params['E12'])
    E13 = np.array(params['E13'])
    E31 = np.array(params['E31'])
    E32 = np.array(params['E32'])
    E10 = np.array(params['E10'])
    E41 = np.array(params['E41'])
    E42 = np.array(params['E42'])
    #for a,b in params.items():
        #a = params[f"{a}"]
    specification = params['specification']

    # Calculate k, gamma, alpha, a, s_ge, delta, beta
    log_k = expr[specification]['log_k'](E11, E12)
    log_gamma = expr[specification]['log_gamma'](log_k, E11)
    log_s = expr[specification]['log_s'](log_gamma, E13, log_k)
    k = np.exp(log_k)
    g = np.exp(log_gamma)
    s = np.exp(log_s)
    EG31 = expr[specification]['EG31'](E31,g)
    EG32 = expr[specification]['EG32'](E32,g)
    EG10 = expr[specification]['EG10'](E10,g)
    EG41 = expr[specification]['EG41'](E41,g)
    EG42 = expr[specification]['EG42'](E42,g)
    alpha = 100/9*k*(EG32-EG31)
    a = 100*k*EG31-100*s-alpha
    s_ge = k*EG10 - s
    delta = np.sqrt((k*EG42-s)/(k*EG41-s))
    beta  = 100*(k*EG41-s)/(delta**2)
    return k, g, s, alpha, a, s_ge, beta, delta



In [8]:
#Define the dictionary to put into function
#Estimate the result
##Table 5 minimum distance estimates: columns (1) (3) panel A and columns (1) (4) panel B

params = {'E11':emp_moments[0],'E12':emp_moments[1],'E13':emp_moments[2],'E31':emp_moments[6],'E32':emp_moments[7],
          "E10":emp_moments[4],'E41':emp_moments[8],'E42':emp_moments[9],'specification':'Exp'}
#Including the mean effort from empirical data for each treatment.

Table5Exp = np.array(mymindisest(params)).flatten()
params['specification'] = 'Power'
Table5Power = np.array(mymindisest(params)).flatten()
Table5Exp


array([1.26689239e-16, 1.57711308e-02, 3.31549170e-06, 3.07335179e-03,
       1.42605422e-01, 8.57890628e-06, 1.15234320e+00, 7.64690724e-01])

## 3. Estimation

### Point Estimates and Standard Errors

We now compute the minimum distance estimates for Table 5 and the standard errors via a bootstrap procedure.


In [9]:
vmindisest = np.vectorize(mymindisest)
params['specification'] = "Exp"
estimatesExp = vmindisest(params)
#mean_sd = np.zeros((8,2))
#for i in range(0,8):
    #mean_sd[i,0],mean_sd[i,1] = np.nanmean(estimatesExp[i]), np.nanstd(estimatesExp[i])

In [10]:
#Define the new dictionary, now ,for each treatment, there is a array that length equal to 2000, which we obtain from boostrap process.
nw_params = {"E11":E11,"E12":E12,"E13":E13,"E31":E31,"E32":E32,"E10":E10,"E41":E41,"E42":E42,"specification":"Exp"}
vmindisest = np.vectorize(mymindisest)#vectorize the function

#Obtain the estimation result
estimatesExp = vmindisest(nw_params)
nw_params['specification'] = 'Power'
estimatesPower = vmindisest(nw_params)

C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:24: RuntimeWarning: overflow encountered in power
  'EG31': lambda E31, g: E31 ** g,'EG32': lambda E32, g: E32 ** g,'EG10': lambda E10, g: E10 ** g,'EG41': lambda E41, g: E41 ** g,'EG42': lambda E42, g: E42 ** g
C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:53: RuntimeWarning: invalid value encountered in subtract
  alpha = 100/9*k*(EG32-EG31)
C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:54: RuntimeWarning: invalid value encountered in multiply
  a = 100*k*EG31-100*s-alpha
C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:55: RuntimeWarning: invalid value encountered in multiply
  s_ge = k*EG10 - s
C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:56: RuntimeWarning: invalid value encountered in multiply
  delta = np.sqrt((k*EG42-s)/(k*EG41-s))
C:\Users\ua896\AppData\Local\Temp\ipykernel_62512\2675283967.py:56: RuntimeWarning: invalid value encountered in tru

In [11]:
# Nowe, to obtain confidence intervals in panel B table 5. CI are the 2.5% and 97.5% quantiles of the distribution of our
# parameters vectors. There are 2000 values in each vector. The 2.5% number and 97.5% number is constant in the original code.
# However, we may obtain some numbers equal to nan or inf when processing, which may affect on our estimation result.
#Therefore, I modify the code, making it can be change accordingly. In this part, We obtain CI only for alpha, a, s_ge, beta, delta as in the paper

CI_Exp, CI_Power = [], []
for ci in range(3,8):
    a  = sorted(estimatesExp[ci][~np.isnan(estimatesExp[ci])&~np.isinf(estimatesExp[ci])])
    CI_Exp.append([a[round(len(a)*0.025)],a[round(len(a)*0.975)]])
    b  = sorted(estimatesPower[ci][~np.isnan(estimatesPower[ci])&~np.isinf(estimatesPower[ci])])
    CI_Power.append([b[round(len(b)*0.025)],b[round(len(b)*0.975)]])

In [12]:
#Function for compute the mean and standard error of estimates for the cost function using the Bootstrap procedure
def cal_res(params,type):
    
    '''params : params is a dictionary, which include each bootstraping result for each parameters
       type : Type can be Exp or Power, depends on whcih cost function you want to apply.'''
    if type == "Exp":
        params['specification'] = "Exp"
        estimates = vmindisest(params)
    else:
        params['specification'] = "Power"
        estimates = vmindisest(params)
    res = np.zeros((8,2))
    for i in range(0,8):
        res[i,0],res[i,1] = np.mean(estimates[i][~np.isnan(estimates[i])&~np.isinf(estimates[i])]), np.std(estimates[i][~np.isnan(estimates[i])&~np.isinf(estimates[i])])
    return res


In [13]:
# Store mean and standard error of estimates for the cost function using the Bootstrap procedure
warnings.filterwarnings('ignore') # This is to avoid showing RuntimeWarning in the notebook regarding overflow. For a couple of cases in our 1000 new samples
                                  # we cannot find the results because of overflow. Losing 2-3 observations out of thousands should not change the overall mean
                                  # for the parameters
exp_res = cal_res(nw_params,type="Exp")
power_res = cal_res(nw_params,type="Power")

In [14]:
params_name = ["Level k of cost of effort", "Curvature γ of cost function","Intrinsic motivation s","Social preferences α",
                "Warm glow coefficient a","Gift exchange Δs", "Present bias β","(Weekly) discount factor δ"]
CIpanelB = pd.DataFrame({'CI_Exp':CI_Exp, 'CI_Power':CI_Power})
sd_exp   = exp_res[0:8,1]
sd_power = power_res[0:8,1]

In [20]:
mini_dist_power_author = ['{0:.2e}'.format(Decimal(2.56E-112)),round(33.137,3),'{0:.2e}'.format(Decimal(7.13E-07)),
                          round(0.003,3), round(0.125,3), '{0:.2e}'.format(Decimal(3.26e-06)), round(1.17,2), round(0.75,2)]

mini_dist_exp_author = ['{0:.2e}'.format(Decimal(1.27E-16)),round(0.0158,3),'{0:.2e}'.format(Decimal(3.32E-06)),
                          round(0.003,3), round(0.143,3), '{0:.2e}'.format(Decimal(8.59e-06)), round(1.15,2), round(0.76,2)]

In [28]:
#I did not change the code in this cell
# Print the results
# Formatting the results nicely for the table

from decimal import Decimal

columns = [Table5Power, sd_power, Table5Exp, sd_exp]
vs = []
for col in columns:
    col = ['{0:.2e}'.format(Decimal(col[0])), round(col[1],3), '{0:.2e}'.format(Decimal(col[2])),
           round(col[3],3), round(col[4],3), '{0:.2e}'.format(Decimal(col[5])), round(col[6],2), round(col[7],2)]
    vs.append(col)
    
Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort power cost function estimates': vs[0],
                              'Minimum dist est on average effort power cost function from authors': mini_dist_power_author,
                              'Minimum dist est on average effort power standard errors': vs[1],
                              'Minimum dist est on average effort exp cost function estimates': vs[2],
                              'Minimum dist est on average effort exp cost function from authors': mini_dist_exp_author,
                              'Minimum dist est on average effort exp standard errors': vs[3]})
    
# Standard errors are different since the seed we used for the bootstrap procedure is different from the one used by the authors since 
# random generation across softwares/languages is not easily replicated (each software uses its own algorithm)

from IPython.display import display
print('Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates')
display(Table5Results)

Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates


,Parameters name,Minimum dist est on average effort power cost function estimates,Minimum dist est on average effort power cost function from authors,Minimum dist est on average effort power standard errors,Minimum dist est on average effort exp cost function estimates,Minimum dist est on average effort exp cost function from authors,Minimum dist est on average effort exp standard errors
0,Level k of cost of effort,2.54e-112,2.56e-112,1.16e-61,1.27e-16,1.27e-16,2.10e-11
1,Curvature γ of cost function,33.138,33.137,11.41,0.016,0.016,0.005
2,Intrinsic motivation s,7.12e-7,7.13e-7,1.10e-5,3.32e-6,3.32e-6,2.47e-5
3,Social preferences α,0.003,0.003,0.013,0.003,0.003,0.013
4,Warm glow coefficient a,0.125,0.125,0.144,0.143,0.143,0.147
5,Gift exchange Δs,3.26e-6,3.26e-6,2.48e-5,8.58e-6,8.59e-6,3.81e-5
6,Present bias β,1.17,1.17,3.02,1.15,1.15,2.76
7,(Weekly) discount factor δ,0.75,0.75,0.31,0.76,0.76,0.31


In [29]:
Table5Results.filter(regex= "power cost function").iloc[1,:]

Minimum dist est on average effort power cost function estimates       33.138
Minimum dist est on average effort power cost function from authors    33.137
Name: 1, dtype: object

In [30]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows = 3, cols= 2, shared_yaxes=False)#subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting')
                        
trace1 = go.Bar(
        x=['my gamma estimation','authors estimation'],
        y=Table5Results.filter(regex= "power cost function").iloc[1,:],
        name='gamma estimation comparison',
        textposition='auto',
        text=Table5Results.filter(regex= "power cost function").iloc[1,:]
    )

trace2 = go.Bar(
        x=['my Social preferences estimation','author estimation'],
        y=Table5Results.filter(regex= "power cost function").iloc[3,:],
        name='Social preferences estimation comparison',
        textposition='auto',
        text=Table5Results.filter(regex= "power cost function").iloc[3,:]
    )

trace3 = go.Bar(
        x=['my coefficient a estimation','author estimation'],
        y=Table5Results.filter(regex= "power cost function").iloc[4,:],
        name='Warm glow coefficient a estimation comparision',
        textposition='auto',
        text=Table5Results.filter(regex= "power cost function").iloc[4,:]
    )

trace4 = go.Bar(
        x=['my Present bias β estimation','author estimation'],
        y=Table5Results.filter(regex= "power cost function").iloc[6,:],
        name='Present bias β estimation comparision',
        textposition='auto',
        text=Table5Results.filter(regex= "power cost function").iloc[6,:]
    )

trace5 = go.Bar(
        x=['discount factor δ estimation','author estimation'],
        y=Table5Results.filter(regex= "power cost function").iloc[7,:],
        name='Discount factor δ estimation comparision',
        textposition='auto',
        text=Table5Results.filter(regex= "power cost function").iloc[7,:]
    )


    #fig.append_trace(trace3, 1,1)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace4,2,1)
fig.append_trace(trace3,2,2)
fig.append_trace(trace5,3,1)



    # Change the bar mode
fig.update_layout(barmode='group',
        title='Comparison of' + " "+ f'{a}'+" "+ 'function parameters between my result and the result of authors')

In [16]:
nw_params['specification'] = "Power"
estimatesPower = vmindisest(nw_params)
exp_result = np.zeros((8,2))
for i in range(0,8):
    exp_result[i,0],exp_result[i,1] = np.mean(estimatesExp[i][~np.isnan(estimatesExp[i])&~np.isinf(estimatesExp[i])]), np.std(estimatesExp[i][~np.isnan(estimatesExp[i])&~np.isinf(estimatesExp[i])])
exp_result

array([[1.48760152e-12, 2.10118455e-11],
       [1.69019075e-02, 5.40929054e-03],
       [1.23427774e-05, 2.46538168e-05],
       [3.73428488e-03, 1.31383155e-02],
       [1.76396374e-01, 1.46980716e-01],
       [1.99015147e-05, 3.80894975e-05],
       [1.92913390e+00, 2.76006686e+00],
       [8.05613488e-01, 3.06057998e-01]])

In [17]:
a = np.array(nw_params['E10'])
b = np.array(nw_params['E11'])
al = 100/9*1.5*(a-b)
al

array([-7300.        , -7433.33333333, -7350.        , ...,
       -7766.66666667, -7800.        , -6900.        ])

In [18]:
a_list = []
re = ["k", "g", "s", "alpha", "a", "s_ge", "beta", "delta"]
for i in range(0,8):
    a_list.append(i)
for i,j in zip(a_list,re):
    a,b = f'{j}'+"_exp"+"_meann", f'{j}'+"_exp"+"_sdd"
    a,b= np.nanmean(estimatesExp[i]), np.nanstd(estimatesExp[i])


In [19]:
import pandas as pd
df_test = pd.read_pickle("D:\\Temp\\epp_final\\bld\\python\\data\\data_modify.pickle")
df_test

,buttonpresses,treatment,treatmentname,payoff_per_100,payoff_charity_per_100,dummy_charity,delay_wks,delay_dummy,gift_dummy,prob,weight_dummy,buttonpresses_nearest_100,logbuttonpresses_nearest_100,dummy1,samplenw,samplepr
0,45.1,1.1,1c PieceRate,0.01,0.0,0,0,0,0,1.0,0,25.0,3.218876,1,1,1
1,100.1,1.1,1c PieceRate,0.01,0.0,0,0,0,0,1.0,0,100.0,4.605170,1,1,1
2,114.1,1.1,1c PieceRate,0.01,0.0,0,0,0,0,1.0,0,100.0,4.605170,1,1,1
3,115.1,1.1,1c PieceRate,0.01,0.0,0,0,0,0,1.0,0,100.0,4.605170,1,1,1
4,126.1,1.1,1c PieceRate,0.01,0.0,0,0,0,0,1.0,0,100.0,4.605170,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9856,3031.1,10,Gift Exchange,0.00,0.0,0,0,0,1,1.0,0,3000.0,8.006368,0,1,0
9857,3060.1,10,Gift Exchange,0.00,0.0,0,0,0,1,1.0,0,3100.0,8.039157,0,1,0
9858,3073.1,10,Gift Exchange,0.00,0.0,0,0,0,1,1.0,0,3100.0,8.039157,0,1,0
9859,3135.1,10,Gift Exchange,0.00,0.0,0,0,0,1,1.0,0,3100.0,8.039157,0,1,0
